In [1]:
from fixtures import calculate_team_stats, get_next_opponents, opponents_to_dataframe, opponents_to_dataframe_def, stats_to_dataframe
from fetcher import get_teams_dict, fetch_fixtures
import pandas as pd
import numpy as np

In [2]:
# Fetch the fixture data
fixtures = fetch_fixtures()
teams = get_teams_dict()

In [3]:
CURRENT_GAMEWEEK = 7
NEXT_GAMEWEEK = 4
PREV_GAMEWEEK = 4

In [4]:
stats = calculate_team_stats(fixtures, CURRENT_GAMEWEEK, PREV_GAMEWEEK)

# Convert to DataFrame
df = stats_to_dataframe(stats, teams)

In [5]:
# Define the range and number of clusters
range_min, range_max = df['Points'].min(), df['Points'].max()
goal_range_min, goal_range_max = df['Goals Scored'].min(), df['Goals Scored'].max()
conceded_range_min, conceded_range_max = df['Goals Conceded'].min(), df['Goals Conceded'].max()
num_clusters = 5

# Generate evenly spaced data points for the clusters
points_bracket = np.linspace(range_min, range_max, num_clusters)
goals_bracket = np.linspace(goal_range_min, goal_range_max, num_clusters)
conceded_bracket = np.linspace(conceded_range_min, conceded_range_max, num_clusters)

bracket_labels = [1, 2, 3, 4]
bracket_labels_rev = [4, 3, 2, 1]

df['point_bracket'] = pd.cut(df['Points'], bins=points_bracket, labels=bracket_labels, include_lowest=True)
df['goals_bracket'] = pd.cut(df['Goals Scored'], bins=goals_bracket, labels=bracket_labels, include_lowest=True)
df['conceded_bracket'] = pd.cut(df['Goals Conceded'], bins=conceded_bracket, labels=bracket_labels_rev, include_lowest=True)
df[["Team", "point_bracket", "goals_bracket", "conceded_bracket"]]

,Team,point_bracket,goals_bracket,conceded_bracket
17,Arsenal,4,4,3
15,Chelsea,4,4,4
8,Liverpool,4,2,4
10,Man City,4,3,3
12,Aston Villa,4,3,3
6,Fulham,3,3,3
16,Spurs,3,3,3
1,Man Utd,2,1,4
13,Everton,2,2,3
19,Newcastle,2,1,3


In [6]:
id_point_bracket_map = df.set_index('ID')['point_bracket'].to_dict()

In [7]:
# Next GW
opponents = get_next_opponents(fixtures, CURRENT_GAMEWEEK, 1)

df_opp = opponents_to_dataframe(opponents, id_point_bracket_map, teams)
df_opp = df_opp.sort_values(by=["opp_diff", "diff"], ascending=[True, False])
opp_field = [f"next_{i}_opp" for i in range(1, 2)]
field = ["team_name"]
field = field + (opp_field)
field.append("diff")
field.append("opp_diff")

df_opp[field]

,team_name,next_1_opp,diff,opp_diff
18,Nott'm Forest,Crystal Palace h,2,0.5
15,Man City,Wolves a,4,1.0
0,Spurs,West Ham h,3,1.0
4,Ipswich,Everton h,2,1.0
6,Man Utd,Brentford h,2,1.0
8,Newcastle,Brighton h,2,1.0
11,Leicester,Southampton a,2,1.0
10,Southampton,Leicester h,1,1.0
13,Arsenal,Bournemouth a,4,1.5
5,Everton,Ipswich a,2,1.5


In [8]:
# Next 4 GW, Custom FDR
opponents = get_next_opponents(fixtures, CURRENT_GAMEWEEK, NEXT_GAMEWEEK)

df_opp = opponents_to_dataframe(opponents, id_point_bracket_map, teams)
df_opp = df_opp.sort_values(by=["opp_diff", "diff"], ascending=[True, False])
opp_field = [f"next_{i}_opp" for i in range(1, NEXT_GAMEWEEK+1)]
field = ["team_name"]
field = field + (opp_field)
field.append("diff")
field.append("opp_diff")

df_opp[field]

,team_name,next_1_opp,next_2_opp,next_3_opp,next_4_opp,diff,opp_diff
18,Nott'm Forest,Crystal Palace h,Leicester a,West Ham h,Newcastle h,2,1.6
15,Man City,Wolves a,Southampton h,Bournemouth a,Brighton a,4,1.8
14,Wolves,Man City h,Brighton a,Crystal Palace h,Southampton h,1,1.8
0,Spurs,West Ham h,Crystal Palace a,Aston Villa h,Ipswich h,3,2.0
11,Leicester,Southampton a,Nott'm Forest h,Ipswich a,Man Utd a,2,2.0
2,Fulham,Aston Villa h,Everton a,Brentford h,Crystal Palace a,3,2.2
1,West Ham,Spurs a,Man Utd h,Nott'm Forest a,Everton h,2,2.2
4,Ipswich,Everton h,Brentford a,Leicester h,Spurs a,2,2.2
5,Everton,Ipswich a,Fulham h,Southampton a,West Ham a,2,2.2
6,Man Utd,Brentford h,West Ham a,Chelsea h,Leicester h,2,2.2


In [9]:
# Next 4 GW, Default FDR
df_opp = opponents_to_dataframe_def(opponents, teams)
df_opp = df_opp.sort_values(by=["opp_diff"], ascending=[True])
opp_field = [f"next_{i}_opp" for i in range(1, NEXT_GAMEWEEK+1)]
field = ["team_name"]
field = field + (opp_field)
field.append("opp_diff")

df_opp[field]

,team_name,next_1_opp,next_2_opp,next_3_opp,next_4_opp,opp_diff
18,Nott'm Forest,Crystal Palace h,Leicester a,West Ham h,Newcastle h,2.0
6,Man Utd,Brentford h,West Ham a,Chelsea h,Leicester h,2.0
0,Spurs,West Ham h,Crystal Palace a,Aston Villa h,Ipswich h,2.2
15,Man City,Wolves a,Southampton h,Bournemouth a,Brighton a,2.2
5,Everton,Ipswich a,Fulham h,Southampton a,West Ham a,2.2
19,Crystal Palace,Nott'm Forest a,Spurs h,Wolves a,Fulham h,2.2
7,Brentford,Man Utd a,Ipswich h,Fulham a,Bournemouth h,2.4
11,Leicester,Southampton a,Nott'm Forest h,Ipswich a,Man Utd a,2.4
14,Wolves,Man City h,Brighton a,Crystal Palace h,Southampton h,2.4
4,Ipswich,Everton h,Brentford a,Leicester h,Spurs a,2.4
